### Requesting force majeure withdrawals:

In [1]:
from tqdm import tqdm
import json
import requests
import pandas as pd
import time
from urllib.parse import urlencode
from config import JUSTER_ADDRESS


def make_deep_requests(url, depth=5, limit=1000, sleep_time=2, stop_id=None):
    """ Copypasted from https://github.com/ztepler/
        quipuswap-tezos-analysis-colab/blob/main/
        QuipuSwap_Tezos_Pool_Analysis.ipynb"""

    last_id = None
    data = pd.DataFrame()

    for counter in tqdm(range(depth)):
        request_url = f'{url}&lastId={last_id}' if last_id else url

        try:
            response = requests.get(request_url)
            new_data = pd.json_normalize(json.loads(response.text))
            data = data.append(new_data)

        except Exception as exception:
            print(exception)
            import pdb; pdb.set_trace()

        last_id = data.id.iloc[-1]

        if len(new_data) < limit:
            break

        if stop_id in data['id'].values:
            break

        time.sleep(sleep_time)

    return data

In [2]:
method = f'https://api.florencenet.tzkt.io/v1/accounts/{JUSTER_ADDRESS}/operations?'
params = urlencode({
    'type': 'transaction',
    'limit': 1000,
    'entrypoint': 'triggerForceMajeure',
    'status': 'applied'
})

data = make_deep_requests(method + params, depth=20)

 20%|██        | 4/20 [00:11<00:46,  2.89s/it]


In [3]:
len(data)

4512

In [4]:
calls = pd.DataFrame({
    'calls_count': data['parameter.value'].value_counts()
})
calls.index.name = 'event_id'
calls.index = calls.index.astype(int)

### Adding info about events current fees:

In [50]:
from sgqlc.endpoint.http import HTTPEndpoint
from config import DIPDUP_ENDPOINT_URI
from executors.force_majeure_caller import detect_failed_start, detect_failed_close

def dirty_dipdup_query(offset=0):
    """ This is one-time used code and I don't like how it written """

    query = """
    query Query {
      juster_event(where: {
        positions: {withdrawn: {_eq: false}},
        closed_oracle_time: {_is_null: true},
        status: {_eq: "CANCELED"}
        }, 
        offset: """ + str(offset) + """
      ) {
        id
        closed_oracle_time
        measure_oracle_start_time
        bets_close_time
        measure_period
      }
    }
    """

    endpoint = HTTPEndpoint(DIPDUP_ENDPOINT_URI)
    dipdup = endpoint(query)
    events = dipdup['data']['juster_event']

    return events


# I know that there are only 107 events so I am doint this,
# very bad, but I need to resolve this issue and go next:
events = dirty_dipdup_query(0) + dirty_dipdup_query(100)
df_events = pd.DataFrame(events)
len(df_events)

107

In [51]:
# calls
df = pd.concat([
    calls,
    df_events.set_index('id')
], axis=1)

In [52]:
assert df['calls_count'].isna().sum() == 0
assert df.index.duplicated().sum() == 0

### 4 events with no info from dipdup that was called only once:
- maybe because all positions are withdrawn?

In [53]:
no_dipdup_info = df['measure_period'].isna()
df[ no_dipdup_info ]

,calls_count,closed_oracle_time,measure_oracle_start_time,bets_close_time,measure_period
0,1,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN
9,1,NaN,NaN,NaN,NaN


### Excluding this objects:

In [54]:
df = df[ ~no_dipdup_info ]

### Calculating required sum:

In [60]:
df['open_fee'] = df['measure_oracle_start_time'].isna() * 0.1
df['close_fee'] = df['closed_oracle_time'].isna() * 0.1

In [62]:
df['total_fee'] = df.apply(
    lambda row: row.calls_count * (row.open_fee + row.close_fee),
    axis=1)

In [63]:
df.total_fee.sum()

680.4